In [1]:
import numpy as np, pandas as pd, bs4

In [2]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

In [3]:
myurl = 'https://web-as.tamu.edu/gradereports/'
registrar_page = uReq(myurl)
page_html = registrar_page.read()
registrar_page.close()
page_soup = soup(page_html, 'html.parser')
diff_distributions = page_soup.findAll('table', {})

7

In [124]:
index = []
years = []
colleges = []
sems = []
for i in range(3):
    index.append([])
    for row in diff_distributions[i+1].findAll('option'):
        index[i].append(row['value'])
    years.append(index[i][:index[i].index('A')])
    colleges.append(index[i][index[i].index('C')+1:])
    sems.append(index[i][index[i].index('A'):index[i].index('C') +1])


In [127]:
def get_df(year, sem, college): 
    url = "https://web-as.tamu.edu/GradeReports/PDFReports/" + str(year) + str(sem) + "/grd" + str(year) + str(sem) + str(college) + ".pdf"
    print(url)
    try:
        df = tabula.read_pdf(url,pages = 'all')
    except urlopen.httperror:
        return [[],[]]
    cols = ['SECTION', "A",'B', "C", 'D', 'F','TOTAL A-F','GPA','I','S','U','Q','X','TOTAL','INSTRUCTOR' ]
    
    combined_df = pd.DataFrame(columns = cols)
    for i in range(len(df)-1):
        df[i].loc[-1] = df[i].columns.values
        df[i].index = df[i].index + 1  # shifting index
        df[i].sort_index(inplace=True)
        print(cols, df[i].columns)
        df[i].columns = cols
        combined_df = combined_df.append(df[i], ignore_index = True)
    combined_df = combined_df.iloc[::2]
    course_df = combined_df[combined_df['SECTION'] == "COURSE TOTAL:"]
    section_df = combined_df[combined_df['SECTION'] != "COURSE TOTAL:"]
    return course_df, section_df

In [132]:
cols = ['SECTION', "A", 'B', "C", 'D', 'F','TOTAL A-F','GPA','I','S','U','Q','X','TOTAL','INSTRUCTOR' ]
merged_course_df = pd.DataFrame(columns = cols)
merged_section_df = pd.DataFrame(columns = cols)
for year in years[0]:
    for college in colleges[0]:
        if "PROF" in college:
            print(college)
            continue
        for sem_val in range(3):   
            merged_course_df.append(get_df(year, sem_val+1, college)[0], ignore_index = True)
            merged_section_df.append(get_df(year, sem_val+1, college)[1], ignore_index = True)            


https://web-as.tamu.edu/GradeReports/PDFReports/20211/grd20211AE.pdf
https://web-as.tamu.edu/GradeReports/PDFReports/20211/grd20211AE.pdf
https://web-as.tamu.edu/GradeReports/PDFReports/20212/grd20212AE.pdf
['SECTION', 'A', 'B+', 'B', 'C+', 'C', 'D', 'F', 'TOTAL A-F', 'GPA', 'I', 'S', 'U', 'X', 'TOTAL', 'INSTRUCTOR'] Index(['ASCC-289-200', '12', '7', '9', '0', '1', '29', '0.000', '0.1', '0.2',
       '0.3', '0.4', '0.5', '29.1', 'MCCLURE M'],
      dtype='object')


ValueError: Length mismatch: Expected axis has 15 elements, new values have 16 elements

In [82]:
"""
url = "https://web-as.tamu.edu/GradeReports/PDFReports/20211/grd20211DN_PROF.pdf"
import tabula
df = tabula.read_pdf(url,pages = 'all')
"""

In [83]:
cols = ['SECTION', "A", "B+",'B', "C+", "C", 'D', 'F','TOTAL A-F','GPA','I','S','U','X','TOTAL','INSTRUCTOR' ]
combined_df = pd.DataFrame(columns = cols)
for i in range(len(df)-1):
    df[i].loc[-1] = df[i].columns.values
    df[i].index = df[i].index + 1  # shifting index
    df[i].sort_index(inplace=True)
    df[i].columns = cols
    combined_df = combined_df.append(df[i], ignore_index = True)
combined_df = combined_df.iloc[::2]
combined_df.head()

,SECTION,A,B+,B,C+,C,D,F,TOTAL A-F,GPA,I,S,U,X,TOTAL,INSTRUCTOR
0,DDDS-601-300,54,0,35,0.1,14,0.2,3,106,3.292,0.3,0.4,0.5,0.6,106.1,SVOBODA K
2,COURSE TOTAL:,54,0,35,0,14,0,3,106,3.292,0,0,0,0,106,NaN
4,DDDS-602-300,68,0,27,0,11,0,0,106,3.537,0,0,0,0,106,KESTERKE M
6,COURSE TOTAL:,68,0,27,0,11,0,0,106,3.537,0,0,0,0,106,NaN
8,DDDS-603-300,36,0,52,0,12,0,3,103,3.145,2,0,0,0,105,RUEST L


In [85]:
course_df = combined_df[combined_df['SECTION'] == "COURSE TOTAL:"]
section_df = combined_df[combined_df['SECTION'] != "COURSE TOTAL:"]